In [0]:
import torch
import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt 
import torch.nn.functional as F

In [0]:
n_dim = 2
x_train, y_train = make_blobs(n_samples= 80, n_features= n_dim, centers = [[1,1],[-1,-1],[1,-1],[-1,1]], shuffle = True, cluster_std = 0.3)
x_test, y_test = make_blobs(n_samples = 20, n_features = n_dim, centers = [[1,1],[-1,-1],[1,-1],[-1,1]], shuffle = True, cluster_std = 0.3)

In [0]:
def label_map(y_, from_, to_):
  y = np.copy(y_)
  for f in from_:
    y[y_ == f] = to_
  return y 

y_train = label_map(y_train, [0,1], 0)
y_train = label_map(y_train, [2,3], 1)
y_test = label_map(y_test, [0,1], 0)
y_test = label_map(y_test, [2,3], 1)

In [0]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

In [0]:
class NeuralNet(torch.nn.Module):   # 신경망 모듈(torch.nn.Module)을 상속받는 클래스로 정의
  def __init__(self, input_size, hidden_size):
    super(NeuralNet, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.linear_1 = torch.nn.Linear(self.input_size, self.hidden_size)
    self.relu = torch.nn.ReLU()
    self.linear_2 = torch.nn.Linear(self.hidden_size,1)
    self.sigmoid = torch.nn.Sigmoid()
  def forward(self, input_tensor):
    linear1 = self.linear_1(input_tensor)
    relu = self.relu(linear1)
    linear2 = self.linear_2(relu)
    output = self.sigmoid(linear2)
    return output


In [0]:
model = NeuralNet(2,5) # input_size = 2, hidden_size = 5
lr = 0.03
loss = torch.nn.BCELoss()
epochs = 2000
optimizer = torch.optim.SGD(model.parameters(), lr = lr)


In [19]:
for epoch in range(epochs):
  model.train()
  optimizer.zero_grad()
  train_output = model(x_train)
  train_loss = loss(train_output.squeeze(), y_train)
  if epoch % 100 == 0:
    print('Train loss at {} is {}'.format(epoch, train_loss.item()))
  train_loss.backward()
  optimizer.step()


Train loss at 0 is 0.6783139109611511
Train loss at 100 is 0.6012202501296997
Train loss at 200 is 0.5046395063400269
Train loss at 300 is 0.4013705849647522
Train loss at 400 is 0.31782424449920654
Train loss at 500 is 0.25847089290618896
Train loss at 600 is 0.215714693069458
Train loss at 700 is 0.1825922727584839
Train loss at 800 is 0.15626756846904755
Train loss at 900 is 0.13573947548866272
Train loss at 1000 is 0.11952714622020721
Train loss at 1100 is 0.10651693493127823
Train loss at 1200 is 0.09603817015886307
Train loss at 1300 is 0.08731617778539658
Train loss at 1400 is 0.07999901473522186
Train loss at 1500 is 0.07377967238426208
Train loss at 1600 is 0.06841946393251419
Train loss at 1700 is 0.06375174224376678
Train loss at 1800 is 0.05968950316309929
Train loss at 1900 is 0.0560983307659626


In [21]:
model.eval()
test_loss_before = loss(torch.squeeze(model(x_test)), y_test)
print('After Training, test loss is {}'.format(test_loss_before.item()))

After Training, test loss is 0.04683858901262283


In [22]:
torch.save(model.state_dict(), './model.pt')
print('state_dict format of the model: {}'.format(model.state_dict()))

state_dict format of the model: OrderedDict([('linear_1.weight', tensor([[ 1.0217, -1.6512],
        [ 0.8446, -1.3668],
        [-1.9872,  0.9694],
        [-1.4273, -1.7397],
        [-0.9738,  0.3629]])), ('linear_1.bias', tensor([-0.0447, -0.0343, -0.1614, -0.1951,  0.1334])), ('linear_2.weight', tensor([[ 1.8853,  1.4202,  2.0130, -2.0876,  0.6256]])), ('linear_2.bias', tensor([-2.1199]))])


In [23]:
new_model = NeuralNet(2,5)
new_model.load_state_dict(torch.load('./model.pt'))

<All keys matched successfully>

In [24]:
new_model.eval()
print('벡터 [-1,1]이 레이블 1을 가질 확률은 {}'.format(new_model(torch.FloatTensor([-1,1])).item()))

벡터 [-1,1]이 레이블 1을 가질 확률은 0.9881879687309265
